# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Processing...
Done!


Then, we define the model, object function and optimizer that we use to classify.

In [7]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        # initialization
        super(SimpleNet,self).__init__()
        
        # definition of layers -- picture size (28*28)
        
        # channel number = 16
        # Height = 26 (calculation process -> (28-2-1)/1+1=26)
        # Width = 26
        self.layer1 = nn.Sequential(
                nn.Conv2d(1,16,kernel_size=3),     
                nn.BatchNorm2d(16),      # normalization
                nn.ReLU(inplace=True))
       
        # channel number = 32
        # Height = 12
        # Width = 12         
        self.layer2 = nn.Sequential(
                nn.Conv2d(16,32,kernel_size=3),  # 32, 24, 24
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2)) # 32, 12, 12     
        
        # channel number = 64
        # Height = 10
        # Width = 10
        self.layer3 = nn.Sequential(
                nn.Conv2d(32,64,kernel_size=3),  # 64, 10, 10
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True))
                #nn.MaxPool2d(kernel_size=2,stride=2), 
                #nn.Dropout2d(0.25))
            
        # channel number = 128
        # Height = 4
        # Width = 4
        self.layer4 = nn.Sequential(
                nn.Conv2d(64,128,kernel_size=3), # 128, 8, 8
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2))  # 128, 4, 4
                #nn.Dropout2d(0.25))
            
        self.fc = nn.Sequential(
                nn.Linear(128 * 4 * 4,1024),
                nn.ReLU(inplace=True),
                #nn.Dropout2d(0.5),
                nn.Linear(1024,128),
                nn.ReLU(inplace=True),
                nn.Linear(128,10))
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)        
        return x

    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(),1e-1)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)


Next, we can start to train and evaluate!

In [8]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    train_accuracy = 0
    model = model.train()
    
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images = Variable(images)
        labels = Variable(labels)
        
        # forward process
        out = model(images)
        loss = criterion(out,labels)
        
        # backward process
        optimizer.zero_grad()
        loss.backward()
        
        # optimize process
        optimizer.step()
        
        # recording loss
        train_loss += loss.item()
        
        # calculate accuracy
        _,pred = out.max(1)
        num_correct = (pred == labels).sum().item()
        acc = num_correct / images.shape[0]
       
        train_accuracy += acc
    
    test_loss = 0
    test_accuracy = 0
               
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    for images,labels in test_loader:
        images = Variable(images)
        labels = Variable(labels)
        
        out = model(images)
        
        loss = criterion(out,labels)
        
        # recording loss
        test_loss += loss.item()
        
        _ , pred = out.max(1)
        num_correct = (pred==labels).sum().item()
        acc = num_correct / images.shape[0]
        
        test_accuracy += acc
    
    # print the training and testing accuracy of each epoch
    print('Epoch {} Training  Accuracy {} Testing Accuracy {}'.format(
        epoch+1, train_accuracy / len(train_loader),test_accuracy / len(test_loader)))
 


 33%|███▎      | 155/468 [01:00<01:58,  2.64it/s]


 66%|██████▌   | 309/468 [01:59<00:58,  2.70it/s]


 99%|█████████▉| 463/468 [02:57<00:01,  2.57it/s]


100%|██████████| 468/468 [02:59<00:00,  2.61it/s]

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch 1 Training  Accuracy 0.9496861645299145 Testing Accuracy 0.9879807692307693


...


 33%|███▎      | 156/468 [00:59<02:00,  2.59it/s]


 66%|██████▌   | 310/468 [01:58<00:58,  2.69it/s]


 99%|█████████▉| 464/468 [02:56<00:01,  2.62it/s]


100%|██████████| 468/468 [02:58<00:00,  2.62it/s]


Epoch 10 Training  Accuracy 0.9985476762820513 Testing Accuracy 0.9944911858974359


#### Q5:
Please print the training and testing accuracy.